Take the unitary matrices from Cirq for the Givens rotations and compare them against Xanadu's tutorial here: https://pennylane.ai/qml/demos/tutorial_givens_rotations.html#givens-rotations

This notebook helped identify the fact that Cirq's native Givens implementation is defined such that the angle theta is not divided by two prior to being fed to the sine/cosine functions, contrary to Pennylane's implementation which does

In [1]:
import numpy as np
import cirq

In [2]:
# angles taken from https://pennylane.ai/qml/demos/tutorial_givens_rotations.html#givens-rotations
x = -2 * np.arcsin(np.sqrt(1/3))
y = -2 * np.arcsin(np.sqrt(1/2))

In [3]:
# Define first givens rotation
g1 = cirq.givens(x / 2)._unitary_()

In [4]:
np.round(g1, 5)

array([[ 1.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.8165 +0.j,  0.57735-0.j,  0.     +0.j],
       [ 0.     +0.j, -0.57735-0.j,  0.8165 +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.     +0.j,  0.     +0.j,  1.     +0.j]])

In [5]:
# Define second givens rotation
g2 = cirq.givens(y / 2)._unitary_()

In [6]:
np.round(g2, 5)

array([[ 1.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.70711+0.j,  0.70711-0.j,  0.     +0.j],
       [ 0.     +0.j, -0.70711-0.j,  0.70711+0.j,  0.     +0.j],
       [ 0.     +0.j,  0.     +0.j,  0.     +0.j,  1.     +0.j]])

In [7]:
# Create the initial |100> state 
ket100 = np.array([[0] * 8]).T
ket100[4] = [1]

In [8]:
ket100.shape

(8, 1)

In [9]:
ket100

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])

In [ ]:
# The rotations are only defined in the space of two qubits,
# need to do the kronecker product with an identiy matrix to get the right
# sized matrices

# It's tricky defining the matrix for a givens rotation that targets two 
# non-adjacent qubits so the workaround here is to define the givens rotations
# on adjacent qubits, and then use SWAPS to get the third qubit into an adjacent
# position, putting it back afterwards.

In [10]:
g1_3qubit = np.kron(g1, np.eye(2))

In [11]:
g2_3qubit = np.kron(g2, np.eye(2))

In [12]:
swap = np.eye(4)
swap[[1,2]] = swap[[2,1]]

In [13]:
swap

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

In [14]:
swap_3qubit = np.kron(np.eye(2), swap)

In [15]:
swap_3qubit

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [16]:
swap_3qubit @ g2_3qubit @ swap_3qubit @ g1_3qubit @ ket100

array([[0.        +0.00000000e+00j],
       [0.57735027-9.06493304e-17j],
       [0.57735027-1.11022302e-16j],
       [0.        +0.00000000e+00j],
       [0.57735027+0.00000000e+00j],
       [0.        +0.00000000e+00j],
       [0.        +0.00000000e+00j],
       [0.        +0.00000000e+00j]])

In [17]:
# Pennylane definition
def custom_givens(theta):
    return np.array([[1, 0, 0, 0], 
                     [0, np.cos(theta/2), -np.sin(theta/2), 0], 
                     [0, np.sin(theta/2), np.cos(theta/2), 0],
                     [0, 0, 0, 1]])


In [18]:
custom_givens(x)

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.81649658,  0.57735027,  0.        ],
       [ 0.        , -0.57735027,  0.81649658,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [19]:
np.round(g1, 5)

array([[ 1.     +0.j,  0.     +0.j,  0.     +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.8165 +0.j,  0.57735-0.j,  0.     +0.j],
       [ 0.     +0.j, -0.57735-0.j,  0.8165 +0.j,  0.     +0.j],
       [ 0.     +0.j,  0.     +0.j,  0.     +0.j,  1.     +0.j]])